In [ ]:
import ROOT

In [ ]:
file_gamma01 = ROOT.TFile.Open("./gamma_data/gamma01.root")

In questo primo esempio di utilizzo di pyROOT, vediamo innanzitutto l'uso di **file ROOT** (estensione _.root_) per leggere oggetti immagazzinati in memoria.
* [ROOT files](https://root.cern/manual/root_files/) 

In [ ]:
file_gamma01.ls()

In [ ]:
t_test = file_gamma01.Get("Events")

L'oggetto _t_test_ che abbiamo letto dal file è il **TTree**, un semplice **database** definito in ROOT.
* i **campi** del database vengono detti **TBranches** (= rami di un albero)
* i **record** del database vengono detti _entries_ oppure _eventi_ (termine ereditato dalla fisica delle particelle, dove ogni record è di solito una registrazione di un evento, come una collisione tra particelle, l'arrivo di un raggio cosmico ecc.)

Documentazione su TTrees:  
* [ROOT trees](https://root.cern/manual/trees/) 

In [ ]:
t_test.Print()

In [ ]:
c = ROOT.TCanvas()     ## l'equivalente di 'figure' in matplotlib, spazio bianco dove mostrare i grafici

In [ ]:
t_test.Draw("energyFit_Energy")

In [ ]:
c.Draw()

In [ ]:
chain = ROOT.TChain("Events")

In [ ]:
chain.Add("./gamma_data/gamma*.root")

Documentazione su TChain:

* [Tree chains](https://root.cern/manual/trees/#appending-ttrees-as-a-tchain)

In [ ]:
chain.Draw("energyFit_Energy")

In [ ]:
c.Update()

In [ ]:
c_reso_all = ROOT.TCanvas("c_reso_all","Resolutions",1200,600)
                        # (nome, titolo, dimensioni orizzontali in pixel, dimensioni verticali in pixel)

In [ ]:
c_reso_all.Divide(2,1)
c_reso_all.cd(1);   chain.Draw("energyFit_Energy-mc_logEnergy")
c_reso_all.cd(2);   chain.Draw("mc_delAngle >> histo_all(100,0.,0.15)")
                    ## il segno '>>' scrive il risultato dell'operazione in un nuovo oggetto ROOT, di tipo definito nell'argomento n.3 
                    ## default per una sola variabile graficata --> istogramma a una dimensione (ROOT.TH1F, ROOT.TH1D)

In [ ]:
c_reso_all.Draw()

In [ ]:
c1_scatter = ROOT.TCanvas("c1_scatter","Energy scatter plots",1000,1000)

In [ ]:
c1_scatter.Divide(3,3)

In [ ]:
variables = ["angularFit_theta","angularFit_phi","energyFit_Energy","energyFit_MinLike/energyFit_Ndof","angularFit_chiSq/angularFit_Ndof","event_nHit",
             "energyFit_Xmax","rec_CxPE40","sqrt(rec_coreX*rec_coreX + rec_coreY*rec_coreY)"]  

In [ ]:
for var in variables:
    c1_scatter.cd(variables.index(var)+1)  
    chain.Draw(f'energyFit_Energy-mc_logEnergy:{var}')

In [ ]:
c1_scatter.Draw()

In [ ]:
limits = {
    "angularFit_theta" : [0.,1.2],
    "angularFit_phi" : [-3.15,3.15],
    "energyFit_Energy" : [2.,6.],
    "energyFit_MinLike/energyFit_Ndof" : [0.,100.],
    "angularFit_chiSq/angularFit_Ndof" : [0.,8.],
    "event_nHit" : [0.,400.],
    "energyFit_Xmax" : [250.,600.],
    "rec_CxPE40" : [0.,200.],
    "sqrt(rec_coreX*rec_coreX + rec_coreY*rec_coreY)" : [0.,700.]
}    

In [ ]:
c2_scatter = ROOT.TCanvas("c2_scatter","Energy scatter plots with profiles",1000,1000)

In [ ]:
c2_scatter.Divide(3,3)

In [ ]:
for var,lims in limits.items():
    itemNum = variables.index(var)+1
    c2_scatter.cd(itemNum)
    chain.Draw(f'energyFit_Energy-mc_logEnergy:{var} >> histo_{itemNum}(50,{lims[0]},{lims[1]},50,-1.,1.2)')
    thisHisto = ROOT.gDirectory.Get(f'histo_{itemNum}')
    thisHisto.Draw('BOX')
    thisProfile = thisHisto.ProfileX()
    thisProfile.SetLineColor(ROOT.kRed);   thisProfile.SetLineWidth(3)   
    thisProfile.Draw('E SAME')

Cose da notare:
* Il default per il segno `>>` quando si fa uno scatter plot di due variabili è un istogramma a **due** dimensioni (ROOT.TH2F ecc.)
* Per **profilo** di un istogramma a due dimensioni lungo l'asse x, si intende un grafico dove i punti hanno come ascissa i centri delle classi in quella direzione mentre come ordinata la **media** dei valori in y di quella classe.

Opzioni di disegno degli istogrammi:
* [Opzioni](https://root.cern/doc/v628/classTHistPainter.html#HP01)

In [ ]:
c2_scatter.Draw()

In [ ]:
import numpy as np

pvalue = np.array([0.68])
quantile = np.array([0.])
histo_all = ROOT.gDirectory.Get("histo_all")
histo_all.GetQuantiles(1,quantile,pvalue)

print (f'68% delAngle = {quantile[0]:.4f} radians')

In [ ]:
c3_scatter = ROOT.TCanvas("c3_scatter","DelAngle scatter plots with profiles",1000,1000)

In [ ]:
c3_scatter.Divide(3,3)

In [ ]:
for var,lims in limits.items():
    itemNum = variables.index(var)+1
    c3_scatter.cd(itemNum)
    chain.Draw(f'mc_delAngle:{var} >> histo_{itemNum}(50,{lims[0]},{lims[1]},50,0.,0.3)')
    thisHisto = ROOT.gDirectory.Get(f'histo_{itemNum}')
    thisHisto.Draw('BOX')
    thisProfile = thisHisto.ProfileX()
    thisProfile.SetLineColor(ROOT.kRed);   thisProfile.SetLineWidth(3)   
    thisProfile.Draw('E SAME')

In [ ]:
c3_scatter.Draw()

In [ ]:
c_reso_sel = ROOT.TCanvas("c_reso_sel","Improved resolutions",1200,600)
c_reso_sel.Divide(2,1)

In [ ]:
cuts = {
    "angularFit_theta" : [0.,0.6],
    "angularFit_phi" : [-3.15,3.15],
    "energyFit_Energy" : [2.6,5.2],
    "energyFit_MinLike/energyFit_Ndof" : [0.,70.],
    "angularFit_chiSq/angularFit_Ndof" : [0.6,2.2],
    "event_nHit" : [45.,10000.],
    "energyFit_Xmax" : [250.,600.],
    "rec_CxPE40" : [3.,20000.],
    "sqrt(rec_coreX*rec_coreX + rec_coreY*rec_coreY)" : [0.,600.]
}    

In [ ]:
longcutstring = ''
for var,cutvals in cuts.items() :
    thiscutstring = ' && '+var+' > '+str(cutvals[0])+' && '+var+' < '+str(cutvals[1])
    longcutstring += thiscutstring
longcutstring = longcutstring[4:]    # remove initial &&
print (longcutstring)

In [ ]:
chain.Draw(">> elist", longcutstring, "entrylist")
elist = ROOT.gDirectory.Get("elist")

Come selezionare una parte degli eventi di un TTree/TChain:

* [La TEntryList](https://root.cern/manual/trees/#selecting-a-subset-of-entries-to-be-read)

In [ ]:
chain.SetEntryList(elist)

In [ ]:
c_reso_sel.cd(1)   
chain.Draw("energyFit_Energy-mc_logEnergy")

In [ ]:
c_reso_sel.cd(2)   
chain.Draw("mc_delAngle >> histo_sel(100,0.,0.05)")

In [ ]:
c_reso_sel.Draw()

In [ ]:
histo_sel = ROOT.gDirectory.Get("histo_sel")
histo_sel.GetQuantiles(1,quantile,pvalue)
print (f'68% delAngle = {quantile[0]:.4f} radians')